# Run: Development Testing v0.3.5

```
Lead     : ababaian
Issue    : 
Version  : v0.3.5-dev : ab-dev branch
start    : 2020 08 22
complete : 2020 08 28
files    : ~/serratus/notebook/200822_ab/
s3_files : s3://serratus-public/notebook/200822_ab/
output   : s3://serratus-public/out/200822_dev/
```

### Intro/Objectives

- Development testing to finish diamond / bowtie2 integration


## Initialize Containers

In [ ]:
# Launch EC2 Builder (C5.xlarge)
# Start docker service on amazon linux 2
sudo yum install -y docker
sudo yum install -y git
sudo service docker start

# Download latest serratus repo
git clone https://github.com/ababaian/serratus.git -b diamond-dev
cd serratus/containers

# If you want to upload containers to your repository, include this.
export DOCKERHUB_USER='serratusbio' # optional
sudo docker login # optional

# Build all containers and upload them docker hub repo (if available)
./build_containers.sh

In [ ]:
sudo docker run --rm --entrypoint /bin/bash -it serratus-align:latest

# Initialize Runs

### Protref4a

Pilot protref4 created


In [ ]:
# Download protref4 from Robert with updates from me (re-arrange DV, add RDev)
aws s3 cp s3://serratus-public/sumbler/protref4.fa  ./protref4a.fa 
aws s3 cp s3://serratus-public/sumbler/protref4.msa ./protref4a.msa 

# Make diamond index for protref4
diamond makedb --in protref4a.fa -d protref4a

# Make fasta index for protref4
samtools faidx protref4a.fa
mv protref4a.fa.fai protref4a.sumzer.tsv

md5sum * > protref4a.md5

aws s3 sync ./ s3://serratus-public/seq/protref4a/

In [ ]:
# b849ce4a8ff531e7daf164d01c49551c  protref4a.dmnd
# 2da44e7f53fd1126ee34664700313816  protref4a.fa
# 34ecc814018b826c2b111c407539997d  protref4a.msa
# 3483690c5e0c92d9a08109453ec5258d  protref4a.sumzer.tsv

### Protref4b


In [ ]:
# Download protref4 from Robert with updates from me (re-arrange DV, add RDev)
aws s3 cp s3://serratus-public/seq/protref4b/protref4b.fa  ./
aws s3 cp s3://serratus-public/seq/protref4b/protref4b.msa ./ 

# Make diamond index for protref4
diamond makedb --in protref4b.fa -d protref4b

# Make fasta index for protref4
samtools faidx protref4b.fa
mv protref4b.fa.fai protref4b.sumzer.tsv

md5sum * > protref4b.md5

aws s3 sync ./ s3://serratus-public/seq/protref4b/

# fed5309d58d1eeb1711878ce2e9e9170  protref4b.dmnd
# b978fc1fc3180affaaf40de7b05d4aab  protref4b.fa
# e094fc7db19c07ffcedf8bc42963ab80  protref4b.msa
# 2dfa9417d40fbbc882c433baef77f592  protref4b.sumzer.tsv

### Protref5


In [ ]:
# Download protref4 from Robert with updates from me (re-arrange DV, add RDev)
aws s3 sync s3://serratus-public/rce/protref_v5/ ./

mv protref_v5.fa protref5.fa
mv otus_final.muscle.afa protref5.msa
mv protref_v5_famcounts.tsv protref5.fam.tsv


# Make diamond index for protref5
diamond makedb --in protref5.fa -d protref5

# Make fasta index for protref5
samtools faidx protref5.fa
mv protref5.fa.fai protref5.sumzer.tsv

md5sum * > protref5.md5

aws s3 sync ./ s3://serratus-public/seq/protref5/

# 8b3e3c63dc12fa4837448c2662363c2d  protref5.dmnd
# 9891b06413224a1cfc79b27070ad4498  protref5.fa
# 408665b7c01de9739936860b214577c7  protref5.fam.tsv
# e094fc7db19c07ffcedf8bc42963ab80  protref5.msa
# a996ba01f5b4de8606b632122ebb4c92  protref5.sumzer.tsv

### Dicistroviridae Sequences

Adding a set of dicistroviridae sequences

#### SRA Nucleotide Query

Query: `txid232795[Organism:exp]`
Date : `2020-08-23`
Results: `2871`

Saved to "s3://serratus-public/seq/protref4a/dicistro_all.fa"

In [ ]:
# Reduce Dicistro sequences
# Load serratus-align container on EC2
sudo docker run --rm --entrypoint /bin/bash \
  -it serratus-align:latest
  

In [ ]:
# Dev tools
yum install -y wget tar gzip less vim unzip

# SeqKit Install
wget https://github.com/shenwei356/seqkit/releases/download/v0.12.0/seqkit_linux_amd64.tar.gz &&\
  tar -xvf seqkit* && mv seqkit /usr/local/bin/ &&\
  rm seqkit_linux*
  
# local bedtools install
wget https://github.com/arq5x/bedtools2/releases/download/v2.29.2/bedtools.static.binary
mv bedtools.static.binary bedtools
chmod 755 bedtools; mv bedtools /usr/bin/

# Local usearch install
#The clustered database was made with usearch:

wget https://drive5.com/downloads/usearch11.0.667_i86linux32.gz
gzip -dc usearch11.0.667_i86linux32.gz > usearch
chmod 755 usearch; mv usearch /usr/bin/usearch


In [ ]:
# EMBOSS Installation
yum install gcc-c++ gsl gsl-devel make

# EMBOSS Tools
wget ftp://emboss.open-bio.org/pub/EMBOSS/EMBOSS-6.6.0.tar.gz
tar -xvf EMBOSS-6.6.0.tar.gz
cd EMBOSS-6.6.0/
./configure --without-x && make && make install
cp emboss/msbar /usr/bin/
cd ..

In [ ]:
mkdir -p dicis; cd dicis

aws s3 cp s3://serratus-public/seq/protref4a/dicistro_all.fa ./
aws s3 cp s3://serratus-public/seq/protref4a/dicistro_refseq.fa ./

cat dicistro_refseq.fa dicistro_all.fa > dicistro.fa

# Create a header file to store original seq file
NAME='dicistro'
grep "^>" $NAME.fa > $NAME.headers
gzip $NAME.headers

# Ancient virus: KJ938718.1 added manually
# to ancient.fa

# Remove duplicate and short sequences
# 638
seqkit rmdup -s -i -D $NAME.dup $NAME.fa > $NAME.rmdup.fa

# Remove Accessions shorter than 6000 nt
seqkit seq -m 6000 $NAME.rmdup.fa > $NAME.gt6k.rmdup.fa

# Prune to 95% nucleotide identity
usearch -cluster_smallmem $NAME.gt6k.rmdup.fa \
   -id 0.95 \
   -sortedby other \
   -maxaccepts 4 \
   -maxrejects 64 \
   -maxhits 1 \
   -uc $NAME.id95.uc \
   -centroids $NAME.id95.fa
   
grep "^>" $NAME.id95.fa > $NAME.id95.headers


In [ ]:
# Get ORFs
getorf -sequence $NAME.id95.fa \
  -find 1 \
  -minsize 1000 -reverse N \
  -outseq $NAME.orf.fa
  
getorf -sequence ancient.fa \
  -find 0 \
  -minsize 400 -reverse N \
  -outseq ancient.orf.fa  
# Named ORF1 and ORF2 KJ938718.1

# Sort
#usearch -sortbylength $NAME.orf.fa \
#   -fastaout $NAME.orf.sort.fa
   
# Prune to 95% nucleotide identity
usearch -cluster_smallmem $NAME.orf.fa \
   -id 0.95 \
   -sortedby other \
   -maxaccepts 4 \
   -maxrejects 64 \
   -maxhits 1 \
   -uc $NAME.orf.id95.uc \
   -centroids $NAME.orf.id95.fa

# Seqkit sort
seqkit sort -n $NAME.orf.id95.fa \
  > $NAME.orf.id95.sort.fa

# re-header
#>Coronaviridae.RdRp.NC_038294
sed 's/ \[.*//g' $NAME.orf.id95.sort.fa \
  | sed 's/>\(.*\)_\([0-9]\)\($\)/>Dicistroviridae.ORF\2.\1/g' - \
  > $NAME.ref.fa
  
grep "^>" $NAME.orf.id95.sort.fa ancient.orf.fa > $NAME.protref.headers
gzip $NAME.protref.headers

cat dicistro.ref.fa ancient.orf.fa > dicistro.protref.fa


In [ ]:
aws s3 sync ./ s3://serratus-public/notebook/200822_ab/dicistro/

### Quenyaviridae Sequences

From [Obbard et al](https://academic.oup.com/ve/article/6/1/vez061/5708932)

Downloaded via nuccore `2020 08 28`

```
# Segment 1
KY634871 OR KY634875 OR MN264686 OR MN264681 OR MN371240 OR MN371238 OR MN371244 OR MN371247 OR MN371231
# Segment 1 TSA
GBQS01013140 OR GASG02022301 OR GAUJ01047315 OR GBOS01033164
# Segment 2
KY634872 OR KY634876 OR MN264687 OR MN264682
# Segment 3
KY634873 OR KY634877 OR MN264688 OR MN264683
# Segment 4
MN264684 OR MN371232 
# Segment 5
MH937728 OR MH937729 OR MN264690 OR MN264685 OR MN371233 OR MN371241 OR MN371239 OR MN371245 OR MN371248 OR MN371234 OR MN371235 OR MN371236 OR MN371237 OR MN371243 OR MN371242 OR MN371246 OR MN371249 OR MN371250 OR MN371251 OR MN371252 OR MN371253 OR MN371254
# Segment 5 (RdRp)
GBKE01003710 OR GDIO01031530 OR GEDC01030233 OR GEDC01010646 OR GBVD01020893 OR GBBI01004385 OR GCDX01009922 OR GDDR01018688 OR GDES01014899 OR GCZF01072004 OR GHOY01071622 OR GGPH01203597 OR GHJE01057056 OR GAUC01011130 OR GAVW02027760 OR GCPB01083611 OR GBVI01014741 OR GFJU01178533 OR GBWM01032350 OR HAGY01027881
```



In [ ]:
mkdir -p quenya; cd quenya

# Create a header file to store original seq file
NAME='quenya'
grep "^>" *.fasta > $NAME.headers
gzip $NAME.headers

aws s3 sync s3://serratus-public/notebook/200822_ab/quenya/ ./
rm -rf vez061*
rm queries.txt

# quenya1.fa
# quenya1_tsa.fa
# quenya2.fa
# quenya3.fa
# quenya4.fa
# quenya5.fa
# quenya5_tsa.fa

# Get ORFs
function qorf {
getorf -sequence $1.fasta \
  -find 1 \
  -minsize 1000 -reverse N \
  -outseq $1.orf.fa
  
sed 's/ \[.*//g' $1.orf.fa \
  | sed "s/>\(.*\)_\([0-9]\)\($\)/>Quenyaviridae.$2.\1/g" - \
  >> quenya.ref.fa  
}

qorf quenya1 orf1
qorf quenya2 orf2
qorf quenya3 orf3
qorf quenya4 orf4
qorf quenya5 rdrp

# GetORF from TSA
getorf -sequence quenya1_tsa.fasta \
  -find 1 \
  -minsize 300\
  -outseq quenya1_tsa.orf.fa
  
sed 's/ \[.*//g' quenya1_tsa.orf.fa \
  | sed "s/>\(.*\)_\([0-9]\)\($\)/>Quenyaviridae.orf1.\1/g" - \
  >> quenya.tsa.fa

# GetORF from TSA
getorf -sequence quenya5_tsa.fasta \
  -find 1 \
  -minsize 300 \
  -outseq quenya5_tsa.orf.fa
  
sed 's/ \[.*//g' quenya5_tsa.orf.fa \
  | sed "s/>\(.*\)_\([0-9]\)\($\)/>Quenyaviridae.rdrp.\1/g" - \
  >> quenya.tsa.fa

# Merge genbank + tsa
cat quenya.ref.fa quenya.tsa.fa  > quenya.reftsa.fa


NAME='quenya'
# Prune to 95% nucleotide identity
usearch -cluster_smallmem $NAME.reftsa.fa \
   -id 0.95 \
   -sortedby other \
   -maxaccepts 4 \
   -maxrejects 64 \
   -maxhits 1 \
   -uc $NAME.id95.uc \
   -centroids $NAME.id95.fa

mv quenya.id95.fa quenya.protref.fa

md5sum * > quenya.md5

aws s3 sync  ./ s3://serratus-public/notebook/200822_ab/quenya/

In [ ]:
# Protref5 with quenya update v200828

# Download protref4 from Robert with updates from me (re-arrange DV, add RDev)
aws s3 sync s3://serratus-public/rce/protref_v5/ ./
aws s3 cp s3://serratus-public/notebook/200822_ab/quenya/quenya.protref.fa ./

cat protref_v5.fa quenya.protref.fa > protref5.fa
rm protref_v5.fa quenya.protref.fa

mv otus_final.muscle.afa protref5.msa
mv protref_v5_famcounts.tsv protref5.fam.tsv


# Make diamond index for protref5
diamond makedb --in protref5.fa -d protref5

# Make fasta index for protref5
samtools faidx protref5.fa
mv protref5.fa.fai protref5.sumzer.tsv

md5sum * > protref5.md5

aws s3 sync ./ s3://serratus-public/seq/protref5/

# a89e5190810e309a253ff4d8329b49a6  protref5.dmnd
# 3164cbd0e40b20f32cdd9ca27483492d  protref5.fa
# 408665b7c01de9739936860b214577c7  protref5.fam.tsv
# e094fc7db19c07ffcedf8bc42963ab80  protref5.msa
# fe5d52d6af78ed40c92dcea793a5ca42  protref5.sumzer.tsv


### crASS and Double Jelly Phages

From Anton K:

```
1. TerL.fa ~@~S protein set of TerL (large terminase subunit) of crAss-like phage group.
From https://www.biorxiv.org/content/10.1101/2020.07.20.212944v1.full.pdf
Publicly available from ftp://ftp.ncbi.nih.gov/pub/yutinn/crassfamily_2020/
2. MCP.fa ~@~S protein set of Double Jelly-Roll major capsid proteins (DJR MCP)
From https://pubmed.ncbi.nlm.nih.gov/29636073/
Publicly available from ftp://ftp.ncbi.nih.gov/pub/yutinn/DJR_MCP_2017/
```

Manually updated names of each entry with accessions.

In [ ]:
# Protref5 with phage2 update v200829

# Download protref4 from Robert with updates from me (re-arrange DV, add RDev)
aws s3 sync s3://serratus-public/rce/protref_v5/ ./
aws s3 cp s3://serratus-public/notebook/200822_ab/quenya/quenya.protref.fa ./
aws s3 cp s3://serratus-public/notebook/200830_ak/Koonin_phages/DRJ.sortref.fa ./
aws s3 cp s3://serratus-public/notebook/200830_ak/Koonin_phages/crass.sortref.fa ./

cat protref_v5.fa \
  quenya.protref.fa \
  DRJ.sortref.fa \
  crass.sortref.fa \
  > protref5.fa
  
rm protref_v5.fa quenya.protref.fa DRJ.sortref.fa crass.sortref.fa

mv otus_final.muscle.afa protref5.msa
mv protref_v5_famcounts.tsv protref5.fam.tsv


# Make diamond index for protref5
diamond makedb --in protref5.fa -d protref5

# Make fasta index for protref5
samtools faidx protref5.fa
mv protref5.fa.fai protref5.sumzer.tsv

md5sum * > protref5.md5

aws s3 sync ./ s3://serratus-public/seq/protref5/

#7bb49127fa13c5df69f5ab57a60ae644  protref5.dmnd
#f99c75b070d8f7f915976fa4587d7195  protref5.fa
#408665b7c01de9739936860b214577c7  protref5.fam.tsv
#e094fc7db19c07ffcedf8bc42963ab80  protref5.msa
#fbd749480a96c9f10ecb29dc9f1ceb8b  protref5.sumzer.tsv



# Development Testing

### Initialize local workspace

In [1]:
# Serratus commit version
SERRATUS="/home/artem/serratus"
cd $SERRATUS

# Create local run directory
WORK="$SERRATUS/notebook/200822_ab"
mkdir -p $WORK; cd $WORK

# S3 notebook path
S3_WORK='s3://serratus-public/notebook/200822_ab/'

# date and version
date
git rev-parse HEAD # commit version

Sun Aug 30 10:19:38 PDT 2020
3af0bc63b69402671f33246f2b05a265e448692b


In [ ]:
# SRA Accession Initialize
cd $WORK
BATCH='test_SraRunInfo.csv'

aws s3 cp s3://lovelywater/sra/viro_SraRunInfo.csv.gz ./
gzip -d *

head -n1 viro_SraRunInfo.csv > header

shuf viro_SraRunInfo.csv | head -n 20 > test_runs

cat header test_runs > $BATCH

rm viro_SraRunInfo.csv header test_runs viro_*

In [2]:
# SRA Accession Initialize
cd $WORK
BATCH='test2_SraRunInfo.csv'

aws s3 cp s3://lovelywater/sra/viro_SraRunInfo.csv.gz ./
gzip -d *.gz

head -n1 viro_SraRunInfo.csv > header

shuf viro_SraRunInfo.csv | head -n 1000 > test_runs

cat header test_runs > $BATCH

rm viro_SraRunInfo.csv header test_runs viro_*

download: s3://lovelywater/sra/viro_SraRunInfo.csv.gz to ./viro_SraRunInfo.csv.gz
shuf: write error: Broken pipe
shuf: write error
rm: cannot remove 'viro_SraRunInfo.csv': No such file or directory


In [ ]:
# SRA Accession Initialize
cd $WORK
BATCH='test3_SraRunInfo.csv'

aws s3 cp s3://lovelywater/sra/viro_SraRunInfo.csv.gz ./
gzip -d *.gz

head -n1 viro_SraRunInfo.csv > header

shuf viro_SraRunInfo.csv | head -n 2000 > test_runs

cat header test_runs > $BATCH

rm viro_SraRunInfo.csv header test_runs viro_*

### Terraform Initialize

In [3]:
# For rapid batching; copy out serratus folder
# PROTEIN / DNA MUST BE SET IN CONFIG FILE
# LINE 153
#   options            = "-k ${module.work_bucket.name} -a bowtie2"
#   options            = "-k ${module.work_bucket.name} -a diamond"

TF=$SERRATUS/terraform/main
cd $TF
git diff main.tf
terraform init

# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve

diff --git a/terraform/main/main.tf b/terraform/main/main.tf
index de2d00d..b3bad70 100644
--- a/terraform/main/main.tf
+++ b/terraform/main/main.tf
@@ -12,14 +12,14 @@ variable "aws_region" {
 }
 
 variable "dl_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of downloader nodes (ASG)"
 }
 
 variable "align_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of aligner nodes (ASG)"
 }
 
@@ -38,14 +38,22 @@ variable "dockerhub_account" {
 }
 
 variable "scheduler_port" {
-  type  = number
+  type    = number
   default = 8000
 }
 
+variable "output_bucket" {
+  type = string
+}
+
+//variable "metrics_ip" {
+//  type = string
+//}
+
 // PROVIDER/AWS ##############################
 provider "aws" {
-  version     = "~> 2.49"
-  region      = var.aws_region
+  version = "~> 2.49"
+  region  = var.aws_region
 }
 
 provider "local" {
@@ -81,67 +89,72 @@ re

In [4]:
cd $TF

# Open SSH tunnels to the monitor
./create_tunnels.sh

# If you get an error on port
# run:
# ps aux | grep ssh
# sudo kill <PID of SSH>



Tunnels created:
    localhost:3000 = grafana
    localhost:9090 = prometheus
    localhost:5432 = postgres
    localhost:8000 = scheduler


In [6]:
cd $WORK
BATCH='test2_SraRunInfo.csv'
wc -l $WORK/$BATCH

1001 /home/artem/serratus/notebook/200822_ab/test2_SraRunInfo.csv


In [7]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh $WORK/$BATCH

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/200822_ab/test2_SraRunInfo.csv
  date: Sun Aug 30 10:26:34 PDT 2020
  wc  : 1001 /home/artem/serratus/notebook/200822_ab/test2_SraRunInfo.csv
  md5 : 88bd753808099efbd48f58666a9e51c3  /home/artem/serratus/notebook/200822_ab/test2_SraRunInfo.csv


--------------------------
tmp.chunk00
1001 tmp.chunk00_sraRunInfo.csv
459076ff46b172c8515efd6721a9f50b  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":1000,"total_rows":1000}


 uploadSRA complete.


## Run Serratus

In [5]:
# Set Cluster Parameters =============================
## get Config File (if it doesn't exist)
# curl localhost:8000/config | jq > serratus-config.json

cd $TF
# Make local changes to config file
echo "  Cluster Config File: "
cat serratus-config.json
echo ""
echo ""
# Re-upload config file
curl -T serratus-config.json localhost:8000/config

  Cluster Config File: 
{
  "ALIGN_ARGS": "--very-sensitive-local",
  "ALIGN_MAX_INCREASE": 25,
  "ALIGN_SCALING_CONSTANT": 0.05,
  "ALIGN_SCALING_ENABLE": true,
  "ALIGN_SCALING_MAX": 1000,
  "CLEAR_INTERVAL": 600,
  "DL_ARGS": "",
  "DL_MAX_INCREASE": 10,
  "DL_SCALING_CONSTANT": 0.1,
  "DL_SCALING_ENABLE": true,
  "DL_SCALING_MAX": 250,
  "GENOME": "protref5",
  "MERGE_ARGS": "protein",
  "MERGE_MAX_INCREASE": 5,
  "MERGE_SCALING_CONSTANT": 0.1,
  "MERGE_SCALING_ENABLE": true,
  "MERGE_SCALING_MAX": 15,
  "SCALING_INTERVAL": 120,
  "VIRTUAL_SCALING_INTERVAL": 45
}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0{"ALIGN_ARGS":"--very-sensitive-local","ALIGN_MAX_INCREASE":25,"ALIGN_SCALING_CONSTANT":0.05,"ALIGN_SCALING_ENABLE":true,"ALIGN_SCALING_MAX":1000,"CLEAR_INTERVAL":600,"DL_ARGS":"","DL_MAX_I

### Error handling

In [ ]:
## Stop postgres if it's running 
# systemctl stop postgresql

## Connect to postgres
# psql -h localhost postgres postgres

#  psql -h localhost postgres postgres -c "DELETE FROM blocks WHERE state = 'done';"

### ACCESSION OPERATIONS
## Reset SPLITTING accessions to NEW
# UPDATE acc SET state = 'new' WHERE state = 'splitting';

## Reset SPLIT_ERR accessions to NEW
## (repeated failures can be missing SRA data)
# UPDATE acc SET state = 'new' WHERE state = 'split_err';

## Reset MERGE_ERR accessions to SPLIT_DONE
# UPDATE acc SET state = 'split_done' WHERE state = 'merge_err';

## Clear DONE Accessions (ONLY ON COMPLETION)
# DELETE FROM acc WHERE state = 'merge_done';

### BLOCK OPERATIONS

##  Reset FAIL blocks to NEW
# UPDATE blocks SET state = 'new' WHERE state = 'fail';

# Reset ALIGNING blocks to NEW
# UPDATE blocks SET state = 'new' WHERE state = 'aligning';

# Clear Done
# DELETE FROM blocks WHERE state = 'done';

# RESET STATE
# DELETE FROM blocks WHERE state = 'done';
# DELETE FROM blocks WHERE state = 'fail';
#
#
# DELETE FROM acc WHERE state = 'split_err';
# DELETE FROM acc WHERE state = 'merging';
# DELETE FROM acc WHERE state = 'merge_err';
# DELETE FROM acc WHERE state = 'split_done';


In [ ]:
# Nuke Shutdown
aws ec2 describe-instances \
  --filter Name=tag:Name,Values=serratus-align-instance \
  > align_instances.json

jq '.Reservations[].Instances[].InstanceId' -r align_instances.json \
  | pv -l \
  | xargs -n10 -P10 aws ec2 terminate-instances --instance-ids

In [ ]:
### UPDATE TO PROTREF CAME THROUGH
# SHUTTING DOWN AND RESTARTING AFTER 34K


In [ ]:
# Exploring sumbler data
# DO NOT UNCOMMENT WILL DELETE MANY FILES
# aws s3 sync s3://serratus-public/out/200823_test/sumbler/ ./
# find . -size -2k -exec rm '{}' \;
# grep 'SRAsum' * | sed 's/;/\t  /g' -